# **Web Server Assigment**

# 1




## Overview
In this assignment, you will explore the fundamentals of socket programming for TCP connections in Python. You will learn how to create a socket, bind it to a specific address and port, and send and receive HTTP packets. Additionally, you will gain an understanding of the basic structure of HTTP header formats.

Your task is to develop a web server that processes one HTTP request at a time. The web server should:

1. Accept and parse the incoming HTTP request.
2. Retrieve the requested file from the server’s file system.
3. Construct an HTTP response message comprising the requested file preceded by appropriate header lines.
4. Send the response directly to the client.

If the requested file is not found on the server, the web server must return an HTTP "404 Not Found" message to the client.

## Code
Below is a skeleton code for the web server. Your task is to complete the code. The sections where you need to add your implementation are marked with `#-----#`. Some sections may require multiple lines of code.

## Running the Server
1. Place an HTML file (e.g., `HelloWorld.html`) in the same directory as the server program.
2. Execute the server program.
3. Determine the IP address of the machine running the server (e.g., `128.238.251.26`).
4. From a different machine or the same machine, open a web browser and navigate to the URL corresponding to the server. For example:
   ```
   http://128.238.251.26:6789/HelloWorld.html
   ```
   In this example, `HelloWorld.html` is the name of the HTML file you placed in the server directory. Note the port number `6789` used after the colon. Replace this port number with the one configured in your server code.

5. The browser should display the contents of `HelloWorld.html`. If the port number is omitted (e.g., `http://128.238.251.26/HelloWorld.html`), the browser will default to port 80. In this case, ensure your server is listening on port 80.
6. Test requesting a file that is not present on the server. You should receive a "404 Not Found" message in the browser.


## What to Submit
Submit the following items:

1. The complete server code.
2. Screenshots of your client browser demonstrating:
   - Successful retrieval of the HTML file content from the server.
   - Receiving a "404 Not Found" message for a non-existent file.

---

In [ ]:
#import socket module
from socket import *
import sys # In order to terminate the program

serverPort = 12000
serverSocket = socket(AF_INET, SOCK_STREAM)
hostname=gethostname()
IP=gethostbyname(hostname)
serverSocket.bind((hostname, serverPort))
serverSocket.listen(True)
print(IP)
#Prepare a server socket
#_________#
print ('Ready to serve...')
while True:
	#Establish the connection
	connectionSocket, addr = serverSocket.accept()
	
	try:
		message =connectionSocket.recv(1024).decode()
		print(message)
		if not message:
			continue
		filename = message.split()[1][1:]
		with open(filename, "r") as f:
			outputdata = f.read()
		#Send one HTTP header line into socket
		http_header_line = "HTTP/1.1 200 OK\r\nContent-Type: text/html\r\n\r\n"
		connectionSocket.sendall(http_header_line.encode('utf-8'))		#Send the content of the requested file to the client
		connectionSocket.sendall(outputdata.encode())

		
	except FileNotFoundError:
        # Send HTTP 404 response
		error_message = "HTTP/1.1 404 Not Found\r\n\r\nFile Not Found"
		connectionSocket.sendall(error_message.encode())
		print("File not found error.")
	finally:
		connectionSocket.close()

serverSocket.close()
sys.exit() #Terminate the program after sending the corresponding data

192.168.137.1
Ready to serve...
GET /HellohWorld.html HTTP/1.1
Host: 192.168.137.1:12000
Connection: keep-alive
Cache-Control: max-age=0
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7
Accept-Encoding: gzip, deflate
Accept-Language: en-US,en;q=0.9


File not found error.
GET /HelloWorld.html HTTP/1.1
Host: 192.168.137.1:12000
Connection: keep-alive
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7
Accept-Encoding: gzip, deflate
Accept-Language: en-US,en;q=0.9



GET /HelloWorld.html HTTP/1.1
Host: 1

# 2

Currently, the web server handles only one HTTP request at a time. Implement a multithreaded server
that is capable of serving multiple requests simultaneously. Using threading, first create a main thread
in which your modified server listens for clients at a fixed port. When it receives a TCP connection
request from a client, it will set up the TCP connection through another port and services the client
request in a separate thread. There will be a separate TCP connection in a separate thread for each
request/response pair.

In [ ]:
#import socket module
from socket import *
import threading
import sys  # In order to terminate the program


# Function to handle client requests in a separate thread on a different port
def handle_client(client_port):
    try:
        # Create a new socket for communication on the dynamically allocated port
        thread_socket = socket(AF_INET, SOCK_STREAM)
        thread_socket.bind((serverIP, client_port))
        thread_socket.listen(1)

        print(f"Thread socket listening on port {client_port}...")
        conn, addr = thread_socket.accept()
        print(f"Thread connected to {addr} on port {client_port}.")

        # Receive HTTP request from the client
        message = conn.recv(1024).decode()
        print(f"Received message from {addr}:\n{message}")

        if not message:
            return

        # Parse the filename from the HTTP request
        filename = message.split()[1][1:]  # Remove leading '/'
        print(f"Requested file: {filename}")

        # Open and read the requested file
        with open(filename, "r") as f:
            outputdata = f.read()

        # Send HTTP response header
        http_header_line = "HTTP/1.1 200 OK\r\nContent-Type: text/html\r\n\r\n"
        conn.sendall(http_header_line.encode('utf-8'))

        # Send the file content to the client
        conn.sendall(outputdata.encode())

    except FileNotFoundError:
        # Send HTTP 404 response for file not found
        error_message = "HTTP/1.1 404 Not Found\r\n\r\nFile Not Found"
        conn.sendall(error_message.encode())
        print(f"File not found: {filename}")

    except Exception as e:
        # Log unexpected errors
        print(f"An error occurred: {e}")

    finally:
        # Close the thread socket and the connection
        conn.close()
        thread_socket.close()
        print(f"Connection on port {client_port} closed.")


# Main server logic
serverPort = 12000
serverSocket = socket(AF_INET, SOCK_STREAM)
hostname = gethostname()
serverIP = gethostbyname(hostname)

# Bind the main server socket to the IP and port
serverSocket.bind((serverIP, serverPort))

# Enable the server to listen for incoming connections
serverSocket.listen(5)
print(f"Main server running on IP: {serverIP}, Port: {serverPort}")
print('Ready to serve...')

# Counter to assign unique ports for each client
client_port_counter = serverPort + 1

while True:
    # Accept a new connection from a client
    connectionSocket, addr = serverSocket.accept()
    print(f"Connection established with {addr}")

    # Assign a new port for this client
    client_port = client_port_counter
    client_port_counter += 1

    # Create a new thread to handle the client's request on the new port
    thread = threading.Thread(target=handle_client, args=(client_port,))
    thread.start()


Main server running on IP: 192.168.137.1, Port: 12000
Ready to serve...
Connection established with ('192.168.137.1', 65298)
Thread socket listening on port 12001...
Connection established with ('192.168.137.1', 65299)
Thread socket listening on port 12002...
Connection established with ('192.168.137.1', 65302)
Thread socket listening on port 12003...
Connection established with ('192.168.137.1', 65304)
Thread socket listening on port 12004...
Connection established with ('192.168.137.1', 65305)
Thread socket listening on port 12005...
Connection established with ('192.168.137.1', 65306)
Thread socket listening on port 12006...
Connection established with ('192.168.137.1', 65307)
Thread socket listening on port 12007...
Connection established with ('192.168.137.1', 65308)
Thread socket listening on port 12008...
Thread connected to ('192.168.137.1', 65331) on port 12003.Connection established with ('192.168.137.1', 65334)

Received message from ('192.168.137.1', 65331):
GET /HelloWorld

# 3

The Multi Thread Web Server is now set and working but cannot handle changes in the source files. How can we look for changes in the source directory and access it in your code ? Explore Libraries in python

In [ ]:
from socket import *
from threading import Thread
import sys
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import os

# Cache to store file content
file_cache = {}

# Event handler for file changes
class FileChangeHandler(FileSystemEventHandler):
    def on_modified(self, event):
        if event.is_directory:
            return
        file_path = event.src_path
        if file_path in file_cache:
            print(f"File modified: {file_path}. Updating cache...")
            with open(file_path, "r") as f:
                file_cache[file_path] = f.read()

    def on_deleted(self, event):
        if event.is_directory:
            return
        file_path = event.src_path
        if file_path in file_cache:
            print(f"File deleted: {file_path}. Removing from cache...")
            file_cache.pop(file_path)

# Function to handle client requests
def handle_client(conn, addr):
    try:
        message = conn.recv(1024).decode()
        print(f"Request from {addr}:\n{message}")

        if not message:
            return

        # Parse the requested file name from the HTTP request
        filename = message.split()[1][1:]  # Remove leading '/'
        filepath = os.path.abspath(filename)

        if filepath in file_cache:
            outputdata = file_cache[filepath]
        else:
            try:
                with open(filepath, "r") as f:
                    outputdata = f.read()
                    file_cache[filepath] = outputdata
            except FileNotFoundError:
                error_message = "HTTP/1.1 404 Not Found\r\n\r\nFile Not Found"
                conn.sendall(error_message.encode())
                return

        # Send HTTP response header
        http_header_line = "HTTP/1.1 200 OK\r\nContent-Type: text/html\r\n\r\n"
        conn.sendall(http_header_line.encode('utf-8'))

        # Send the file content to the client
        conn.sendall(outputdata.encode())
        print(f"File {filename} served successfully.")
    except Exception as e:
        print(f"Error handling request from {addr}: {e}")
    finally:
        conn.close()

# Main server logic
def start_server():
    serverPort = 12000
    serverSocket = socket(AF_INET, SOCK_STREAM)
    hostname = gethostname()
    serverIP = gethostbyname(hostname)

    serverSocket.bind((serverIP, serverPort))
    serverSocket.listen(5)
    print(f"Server running on IP: {serverIP}, Port: {serverPort}")
    print('Ready to serve...')

    while True:
        connectionSocket, addr = serverSocket.accept()
        print(f"Connection established with {addr}")
        thread = Thread(target=handle_client, args=(connectionSocket, addr))
        thread.start()

# Start the file watcher
def start_watcher(path_to_watch):
    event_handler = FileChangeHandler()
    observer = Observer()
    observer.schedule(event_handler, path=path_to_watch, recursive=False)
    observer.start()
    print(f"Watching for changes in {path_to_watch}...")
    return observer

if __name__ == "__main__":
    # Directory to watch for file changes
    directory_to_watch = os.getcwd()

    # Start the file watcher
    observer = start_watcher(directory_to_watch)

    try:
        # Start the server
        start_server()
    except KeyboardInterrupt:
        print("Shutting down server and file watcher...")
        observer.stop()
        observer.join()
        sys.exit()


Watching for changes in c:\Users\ASUS\.vscode\.vscode\EEA-Networking101\Week 3 - Web Server...
Server running on IP: 192.168.137.1, Port: 12000
Ready to serve...
Connection established with ('192.168.137.1', 49745)
Request from ('192.168.137.1', 49745):
GET /HelloWorld.html HTTP/1.1
Host: 192.168.137.1:12000
Connection: keep-alive
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7
Accept-Encoding: gzip, deflate
Accept-Language: en-US,en;q=0.9


Connection established with ('192.168.137.1', 49746)
File HelloWorld.html served successfully.
File modified: c:\Users\ASUS\.vscode\.vscode\EEA-Networking101\Week 3 - Web Server\HelloWorld.html. Updating cache...
File modified: c:\Users\ASUS\.vscode\.vscode\EEA-Networking101\Week 3 - Web Server\HelloWorl

After you've accessed it in your code, Post a notification on the hosted http page to reload if any source directory changes

In [32]:
from socket import *
from threading import Thread
import sys
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import os
import asyncio
import websockets

# Cache to store file content
file_cache = {}

# WebSocket clients
websocket_clients = set()

# Event handler for file changes
class FileChangeHandler(FileSystemEventHandler):
    async def notify_clients(self):
        # Notify all connected WebSocket clients
        message = "reload"
        for client in websocket_clients:
            try:
                await client.send(message)
            except:
                pass

    def on_modified(self, event):
        if event.is_directory:
            return
        file_path = event.src_path
        if file_path in file_cache:
            print(f"File modified: {file_path}. Updating cache and notifying clients...")
            with open(file_path, "r") as f:
                file_cache[file_path] = f.read()
            # Schedule notification in the current loop
            asyncio.create_task(self.notify_clients())

    def on_deleted(self, event):
        if event.is_directory:
            return
        file_path = event.src_path
        if file_path in file_cache:
            print(f"File deleted: {file_path}. Removing from cache and notifying clients...")
            file_cache.pop(file_path)
            # Schedule notification in the current loop
            asyncio.create_task(self.notify_clients())

# Function to handle client requests
def handle_client(conn, addr):
    try:
        message = conn.recv(1024).decode()
        print(f"Request from {addr}:\n{message}")

        if not message:
            return

        # Parse the requested file name from the HTTP request
        filename = message.split()[1][1:]  # Remove leading '/'
        filepath = os.path.abspath(filename)

        if filepath in file_cache:
            outputdata = file_cache[filepath]
        else:
            try:
                with open(filepath, "r") as f:
                    outputdata = f.read()
                    file_cache[filepath] = outputdata
            except FileNotFoundError:
                error_message = "HTTP/1.1 404 Not Found\r\n\r\nFile Not Found"
                conn.sendall(error_message.encode())
                return

        # Send HTTP response header
        http_header_line = "HTTP/1.1 200 OK\r\nContent-Type: text/html\r\n\r\n"
        conn.sendall(http_header_line.encode('utf-8'))

        # Send the file content to the client
        conn.sendall(outputdata.encode())
        print(f"File {filename} served successfully.")
    except Exception as e:
        print(f"Error handling request from {addr}: {e}")
    finally:
        conn.close()

# Main server logic
def start_http_server():
    serverPort = 12000
    serverSocket = socket(AF_INET, SOCK_STREAM)
    hostname = gethostname()
    serverIP = gethostbyname(hostname)

    serverSocket.bind((serverIP, serverPort))
    serverSocket.listen(5)
    print(f"HTTP Server running on IP: {serverIP}, Port: {serverPort}")
    print('Ready to serve...')

    while True:
        connectionSocket, addr = serverSocket.accept()
        print(f"Connection established with {addr}")
        thread = Thread(target=handle_client, args=(connectionSocket, addr))
        thread.start()

# WebSocket server logic
async def websocket_handler(websocket, path):
    websocket_clients.add(websocket)
    try:
        async for message in websocket:
            pass  # No client messages expected
    finally:
        websocket_clients.remove(websocket)

async def start_websocket_server():
    server = await websockets.serve(websocket_handler, "0.0.0.0", 8765)
    print("WebSocket server running on ws://0.0.0.0:8765")
    await server.wait_closed()

# Start the file watcher
def start_watcher(path_to_watch):
    event_handler = FileChangeHandler()
    observer = Observer()
    observer.schedule(event_handler, path=path_to_watch, recursive=False)
    observer.start()
    print(f"Watching for changes in {path_to_watch}...")
    return observer

async def main():
    # Directory to watch for file changes
    directory_to_watch = os.getcwd()

    # Start the file watcher
    observer = start_watcher(directory_to_watch)

    # Start HTTP server in a thread
    http_thread = Thread(target=start_http_server)
    http_thread.start()

    # Start WebSocket server
    await start_websocket_server()

    observer.stop()
    observer.join()

if __name__ == "__main__":
    try:
        if sys.platform.startswith('win') and sys.version_info >= (3, 8):
            asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
        if asyncio.get_event_loop().is_running():
            await main()
        else:
            asyncio.run(main())
    except KeyboardInterrupt:
        print("Shutting down servers...")
        sys.exit()


Exception in thread Thread-99 (start_http_server):
Traceback (most recent call last):
  File "c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\ASUS\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_42184\3601255534.py", line 92, in start_http_server
OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted


Watching for changes in c:\Users\ASUS\.vscode\.vscode\EEA-Networking101\Week 3 - Web Server...


OSError: [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8765): only one usage of each socket address (protocol/network address/port) is normally permitted

Your dynamic http server is now ready